# Simple RAG Application with Chroma VectorDB

This notebook demonstrates a basic Retrieval-Augmented Generation (RAG) setup using Chroma as the vector database and OpenAI's embeddings & LLM for querying.

Steps:
1. Install dependencies
2. Load documents
3. Create embeddings and store in Chroma
4. Perform retrieval and generate answers


## 1. Install Dependencies

In [ ]:
# !pip install chromadb langchain-community openai tiktoken 

## 2. Imports and Setup

In [ ]:
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
load_dotenv("../.env")

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'), base_url='https://api.openai.com/v1')

## 3. Load Documents
Replace `'docs/'` with your folder containing text files.

In [ ]:
import glob

doc_paths = glob.glob('../docs/*.txt')
docs = []
for path in doc_paths:
    with open(path, 'r', encoding='utf-8') as f:
        content = f.read()
    docs.append({'text': content, 'metadata': {'source': os.path.basename(path)}})

print(f"Loaded {len(docs)} documents.")

## 4. Create Embeddings and Chroma Vector Store

In [ ]:
# Initialize embeddings
txt_emb = OpenAIEmbeddings()

# Create Chroma vector store
vectordb = Chroma.from_texts(
    [d['text'] for d in docs],
    embedding=txt_emb,
    metadatas=[d['metadata'] for d in docs],
    persist_directory='chroma_db'
)

# Persist to disk
vectordb.persist()
print('Chroma vector store created and persisted.')

## 5. Retrieval-Augmented Generation
Use the vector store to retrieve relevant documents

In [ ]:
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("Tell me more about generative AI")
print(retrieved_docs)

### Use the vector store to answer questions

In [ ]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

print(example_messages[0].content)

### Important information

If you look at [docs/doc1.txt](../docs/doc1.txt), I have added this extra line that `Generative AI for discovered by Dr. Anush Sankaran.`

There is no chance that the LLM would have known this secret (!) by default. However, when you ask the question `Tell me more about generative AI?`, you could see that the answer would include information about `Dr. Anush Sankaran`. 

This demonstrates that LLM retrieves relevant document information from the vectorDB, adds this information to the context, while generating an answer - which is the whole concept of RAG

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | client
    | StrOutputParser()
)

for chunk in rag_chain.stream("Tell me more about generative AI"):
    print(chunk, end="", flush=True)

 Generative AI is a type of algorithm that allows machines to generate new content by learning from existing data. It has various applications, such as content creation and code generation. It was discovered by Dr. Anush Sankaran.